# Homework 3: Neural Networks

## Part 1: Data Preprocessing

### Read data

In [31]:
import pandas as pd

file_path = './amazon_reviews_us_Office_Products_v1_00.tsv'
df = pd.read_csv(file_path, delimiter='\t', on_bad_lines='skip')
df.dropna()

/var/folders/4k/8jk4g7fx3xl9p5mzpl0rkqj40000gp/T/ipykernel_58070/3029288783.py:4: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, delimiter='\t', on_bad_lines='skip')


,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,43081963,R18RVCKGH1SSI9,B001BM2MAC,307809868,"Scotch Cushion Wrap 7961, 12 Inches x 100 Feet",Office Products,5,0.0,0.0,N,Y,Five Stars,Great product.,2015-08-31
1,US,10951564,R3L4L6LW1PUOFY,B00DZYEXPQ,75004341,"Dust-Off Compressed Gas Duster, Pack of 4",Office Products,5,0.0,1.0,N,Y,"Phffffffft, Phfffffft. Lots of air, and it's C...",What's to say about this commodity item except...,2015-08-31
2,US,21143145,R2J8AWXWTDX2TF,B00RTMUHDW,529689027,Amram Tagger Standard Tag Attaching Tagging Gu...,Office Products,5,0.0,0.0,N,Y,but I am sure I will like it.,"Haven't used yet, but I am sure I will like it.",2015-08-31
3,US,52782374,R1PR37BR7G3M6A,B00D7H8XB6,868449945,AmazonBasics 12-Sheet High-Security Micro-Cut ...,Office Products,1,2.0,3.0,N,Y,and the shredder was dirty and the bin was par...,Although this was labeled as &#34;new&#34; the...,2015-08-31
4,US,24045652,R3BDDDZMZBZDPU,B001XCWP34,33521401,"Derwent Colored Pencils, Inktense Ink Pencils,...",Office Products,4,0.0,0.0,N,Y,Four Stars,Gorgeous colors and easy to use,2015-08-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2640249,US,53005790,RLI7EI10S7SN0,B00000DM9M,223408988,PalmOne III Leather Belt Clip Case,Office Products,4,26.0,26.0,N,N,Great value! A must if you hate to carry thing...,I can't live anymore whithout my Palm III. But...,1998-12-07
2640250,US,52188548,R1F3SRK9MHE6A3,B00000DM9M,223408988,PalmOne III Leather Belt Clip Case,Office Products,4,18.0,18.0,N,N,Attaches the Palm Pilot like an appendage,Although the Palm Pilot is thin and compact it...,1998-11-30
2640251,US,52090046,R23V0C4NRJL8EM,0807865001,307284585,Gods and Heroes of Ancient Greece,Office Products,4,9.0,16.0,N,N,"Excellent information, pictures and stories, I...",This book had a lot of great content without b...,1998-10-15
2640252,US,52503173,R13ZAE1ATEUC1T,1572313188,870359649,Microsoft EXCEL 97/ Visual Basic Step-by-Step ...,Office Products,5,0.0,0.0,N,N,class text,I am teaching a course in Excel and am using t...,1998-08-22


### Form classes and sample

In [32]:
cols = ['review_body', 'star_rating']
df = df[cols]

df['review_body'] = df['review_body'].apply(lambda review: '' if pd.isna(review) or isinstance(review, float) else review)

class_1 = df[df['star_rating'].isin([1, 2, 3])]
class_2 = df[df['star_rating'].isin([4, 5])]

sample_size = 50_000
class_1 = class_1.sample(sample_size, random_state=42)
class_2 = class_2.sample(sample_size, random_state=42)

balanced_df = pd.concat([class_1, class_2])
balanced_df.loc[:, 'class'] = balanced_df['star_rating'].apply(lambda x: 1 if x in [1, 2, 3] else 2 if x in [4, 5] else 0)

## Part 2: Word Embedding

### Download gensim Google Word2Vec model

In [4]:
import gensim.downloader as downloader

wv = downloader.load('word2vec-google-news-300')

### Define a basic test

In [11]:
def basic_test(model):
    examples = [
        ('king', 'man', 'woman'),
        ('france', 'paris', 'berlin'),
        ('coffee', 'beans', 'leaves'),
    ]

    for (word1, word2, word3) in examples:
        try:
            result = model.most_similar(positive=[word1, word3], negative=[word2], topn=1)
            print(f'{word1} - {word2} + {word3} = {result[0][0]}')
        except KeyError:
            print(f'"{word1}", "{word2}", or "{word3}" does not exist in this model')

basic_test(wv)

king - man + woman = queen
france - paris + berlin = germany
coffee - beans + leaves = leaving


### Corpus for training model

In [6]:
from gensim import utils

class MyCorpus:
    """An iterator that yields sentences."""

    def __iter__(self):
        data = balanced_df['review_body']
        for line in data:
            yield utils.simple_preprocess(line)

### Train custom model

In [13]:
import gensim.models

sentences = MyCorpus()
model = gensim.models.Word2Vec(sentences=sentences, vector_size=300, window=13, min_count=9, workers=4)
custom_model = model.wv

### Test custom model

In [14]:
basic_test(custom_model)

king - man + woman = synthetic
"france", "paris", or "berlin" does not exist in this model
coffee - beans + leaves = marks


### Custom vs pretrained model analysis

We conclude that my model is much less precise than the pretrained Google model. For example, king - man + woman = oversize, which has no semantic similarity at all.

## Part 3: Simple Models

### Train test split

In [33]:
from sklearn.model_selection import train_test_split

X = balanced_df['review_body']
y = balanced_df['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

### Define average Word2Vec

In [64]:
import numpy as np

def get_average_word2vec(review_string, model, k=300):
    review = review_string.split(' ')
    if (len(review) < 1):
        return np.zeros(k)
    
    embeddings = np.zeros((k,))
    for word in review:
        try:
            embeddings += model[word]
        except KeyError:
            pass
    
    average_embedding = np.divide(embeddings, len(embeddings))

    return average_embedding

### Create test set using Google model

In [72]:
google_model = wv

X_train_w2v = X_train.apply(lambda review: get_average_word2vec(review, google_model))
X_test_w2v = X_test.apply(lambda review: get_average_word2vec(review, google_model))

X_train_w2v = np.array(X_train_w2v.tolist())
X_test_w2v = np.array(X_test_w2v.tolist())

### Create test set using TF-IDF vectorizer

In [67]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

### Test Perceptron

In [69]:
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score

perc_w2v = Perceptron()
perc_w2v.fit(X_train_w2v, y_train)
y_pred_perc_w2v = perc_w2v.predict(X_test_w2v)
accuracy_perc_w2v = accuracy_score(y_test, y_pred_perc_w2v)

perc_tfidf = Perceptron()
perc_tfidf.fit(X_train_tfidf, y_train)
y_pred_perc_tfidf = perc_tfidf.predict(X_test_tfidf)
accuracy_perc_tfidf = accuracy_score(y_test, y_pred_perc_tfidf)

print(f'Perceptron accuracy with Word2Vec: {accuracy_perc_w2v}')
print(f'Perceptron accuracy with TF-IDF: {accuracy_perc_tfidf}')

Perceptron accuracy with Word2Vec: 0.7965
Perceptron accuracy with TF-IDF: 0.82425


### Test SVM

In [71]:
from sklearn.svm import LinearSVC

svm_w2v = LinearSVC(dual='auto')
svm_w2v.fit(X_train_w2v, y_train)
y_pred_svm_w2v = svm_w2v.predict(X_test_w2v)
accuracy_svm_w2v = accuracy_score(y_test, y_pred_svm_w2v)

svm_tfidf = LinearSVC(dual='auto')
svm_tfidf.fit(X_train_tfidf, y_train)
y_pred_svm_tfidf = svm_tfidf.predict(X_test_tfidf)
accuracy_svm_tfidf = accuracy_score(y_test, y_pred_svm_tfidf)

print(f'SVM accuracy with Word2Vec: {accuracy_svm_w2v}')
print(f'SVM accuracy with TF-IDF: {accuracy_svm_tfidf}')

SVM accuracy with Word2Vec: 0.78975
SVM accuracy with TF-IDF: 0.8649


## Part 4: Feedforward Neural Networks